In [1]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "

In [2]:
from datasets import load_dataset

dataset = load_dataset("ethz/food101")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

train-00000-of-00008.parquet:   0%|          | 0.00/490M [00:00<?, ?B/s]

train-00001-of-00008.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

train-00002-of-00008.parquet:   0%|          | 0.00/472M [00:00<?, ?B/s]

...

validation-00000-of-00003.parquet:   0%|          | 0.00/423M [00:00<?, ?B/s]

validation-00001-of-00003.parquet:   0%|          | 0.00/413M [00:00<?, ?B/s]

validation-00002-of-00003.parquet:   0%|          | 0.00/426M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 75750
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 25250
    })
})

In [4]:
from datasets import load_dataset
import tensorflow as tf
from collections import Counter

# 2. Збалансоване обрізання датасету
def balanced_cut(dataset_split, num_classes=101, total_samples=5000):
    samples_per_class = total_samples // num_classes
    indices = []
    class_counter = Counter()

    for idx, label in enumerate(dataset_split['label']):
        if class_counter[label] < samples_per_class:
            indices.append(idx)
            class_counter[label] += 1
        if len(indices) >= samples_per_class * num_classes:
            break

    return dataset_split.select(indices)

small_train_dataset = balanced_cut(dataset['train'], total_samples=5000)
small_val_dataset = balanced_cut(dataset['validation'], total_samples=1000)

In [5]:
# 3. Препроцесинг
def preprocess_data(dataset):
    dataset = dataset.map(lambda x: {
        'image': tf.image.resize(tf.convert_to_tensor(x['image']), (128, 128)) / 255.0,
        'label': x['label']
    }, num_proc=1)
    return dataset

train_data = preprocess_data(small_train_dataset)
val_data = preprocess_data(small_val_dataset)

Map:   0%|          | 0/4949 [00:00<?, ? examples/s]

Map:   0%|          | 0/909 [00:00<?, ? examples/s]

In [6]:
# 4. tf.data.Dataset
train_data = train_data.to_tf_dataset(
    columns=['image'],
    label_cols='label',
    shuffle=True,
    batch_size=32
).prefetch(tf.data.AUTOTUNE)

val_data = val_data.to_tf_dataset(
    columns=['image'],
    label_cols='label',
    shuffle=False,
    batch_size=32
).prefetch(tf.data.AUTOTUNE)

/usr/local/lib/python3.11/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [7]:
# 5. Модель CNN
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(101, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# 6. Колбеки
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
]

# 7. Навчання
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/20
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.0113 - loss: 4.6536

155/155 ━━━━━━━━━━━━━━━━━━━━ 944s 6s/step - accuracy: 0.0113 - loss: 4.6534 - val_accuracy: 0.0099 - val_loss: 4.6150 - learning_rate: 0.0010
Epoch 2/20
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.0153 - loss: 4.6137

155/155 ━━━━━━━━━━━━━━━━━━━━ 979s 6s/step - accuracy: 0.0153 - loss: 4.6137 - val_accuracy: 0.0121 - val_loss: 4.6140 - learning_rate: 0.0010
Epoch 3/20
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.0192 - loss: 4.5907

155/155 ━━━━━━━━━━━━━━━━━━━━ 963s 6s/step - accuracy: 0.0192 - loss: 4.5907 - val_accuracy: 0.0264 - val_loss: 4.5669 - learning_rate: 0.0010
Epoch 4/20
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.0398 - loss: 4.4755

155/155 ━━━━━━━━━━━━━━━━━━━━ 921s 6s/step - accuracy: 0.0398 - loss: 4.4754 - val_accuracy: 0.0297 - val_loss: 4.5052 - learning_rate: 0.0010
Epoch 5/20
155/155 ━━━━━━━━━━━━━━━━━━━━ 908s 6s/step - accuracy: 0.0764 - loss: 4.2024 - val_accuracy: 0.0275 - val_loss: 4.5492 - learning_rate: 0.0010
Epoch 6/20
 96/155 ━━━━━━━━━━━━━━━━━━━━ 4:57 5s/step - accuracy: 0.1556 - loss: 3.6852

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# 8. Графіки
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()

plt.tight_layout()
plt.savefig("training_curves.png")
plt.show()